In [1]:
import hashlib
import json

import string
import random
from bitstring import BitArray
import os
from types import SimpleNamespace

In [2]:
class Block:
    def __init__(self, data='', StartingZeros=0, nonce_len=5):
        self.data = data
        self.nonce_len = nonce_len
        self.StartingZeros = StartingZeros
        self.block_hash = ''
        self.PoW_nonce = ''

    def get_info(self):
        return json.dumps(self.__dict__)
    
    def save(self, path='', name='Block.json'):
        if path != '':   
            with open(os.path.join(path, name), "w") as file:
                file.write(self.get_info())
        else:
            with open(name, "w") as file:
                file.write(self.get_info())
    
    def read(self, path):
        with open(path, "r") as file:
            info = file.read()
            
        self.fill_fields(info)
        return self
    
    def fill_fields(self, info):
        x = json.loads(info, object_hook=lambda d: SimpleNamespace(**d))
        self.data = x.data
        self.nonce_len = x.nonce_len
        self.StartingZeros = x.StartingZeros
        self.block_hash = x.block_hash
        self.PoW_nonce = x.PoW_nonce
        
    def print_info(self):
        print(self.get_info())

In [3]:
class PoW:
        
    def compute_hash(self, block):
        header = block.data + block.PoW_nonce
            
        h = hashlib.sha256()
        h.update(header.encode('utf-8'))
        hash_output = h.hexdigest()

        ba = BitArray(hex=hash_output)
        block_hash = ba.bin  
        
        return block_hash
    
    def compute_PoW(self, block):
        while not block.block_hash.startswith('0' * block.StartingZeros):
            block.PoW_nonce = self.get_random_string(block.nonce_len)
            block.block_hash = self.compute_hash(block)
        return block.block_hash
    
    def get_random_string(self, string_len):
        return ''.join(random.choices(string.ascii_letters + string.digits, k=string_len))
    
    def verify_PoW(self, block, block_hash):
        return (block.block_hash.startswith('0' * block.nonce_len) and block_hash == self.compute_hash(block))

In [4]:
block = Block('message', 17)

In [5]:
Pow = PoW()

In [6]:
Pow.compute_PoW(block)

'0000000000000000001111010111100010101010110111110100010101000100100110010110100101111110001001010101110001101111011100111001101000100001000010011010011111000000100010110000110111101011110110011100010111001100011110011111100000111010000110010010011000001001'

In [7]:
block.print_info()

{"data": "message", "nonce_len": 5, "StartingZeros": 17, "block_hash": "0000000000000000001111010111100010101010110111110100010101000100100110010110100101111110001001010101110001101111011100111001101000100001000010011010011111000000100010110000110111101011110110011100010111001100011110011111100000111010000110010010011000001001", "PoW_nonce": "kl5ZS"}


In [8]:
block.save('/home/maksboruh/course/blockchain/', 'Block.json')

In [9]:
block2 = Block()

In [10]:
block2.print_info()

{"data": "", "nonce_len": 5, "StartingZeros": 0, "block_hash": "", "PoW_nonce": ""}


In [11]:
block2.read('/home/maksboruh/course/blockchain/Block.json')

In [12]:
block2.print_info()

{"data": "message", "nonce_len": 5, "StartingZeros": 17, "block_hash": "0000000000000000001111010111100010101010110111110100010101000100100110010110100101111110001001010101110001101111011100111001101000100001000010011010011111000000100010110000110111101011110110011100010111001100011110011111100000111010000110010010011000001001", "PoW_nonce": "kl5ZS"}


In [13]:
Pow.verify_PoW(block2, block.block_hash)

True

In [14]:
block3 = Block('message', 17)

In [15]:
Pow.compute_PoW(block3)

'0000000000000000010011010101011010111001010111000111010000100110110011001001001010111001010000001110110000111110111011100010110100010101001110001000111001000011001100000100000010100110000011101000100100000100011111011001100011110011101111011111000010110001'

In [16]:
Pow.verify_PoW(block3, block2.block_hash)

False